# Data processing:

some quick notes: due to inconsistent data formatting (& sometimes just wrong dates), i'm currently not getting all the NPIs. i'm always getting around half in each country. Austra is almost perfect by default (go team Austria!). I spent a bit of time making exceptions to get 90% of UK NPIs. I haven't spent any time on making exceptions for Italy or Germany. Expect it to take at most a few hours to get 90+% of all NPIs. If you still plan to do a run tomorrow Mrinank, if you think it's worth it I can probably do a few hours of work in the morn to get up to 90%. lmk

In [ ]:
import math
import os
import copy
import numpy as np
import pandas as pd
import theano
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import black
from datetime import datetime as dt

## UK

Getting the data, extracting regions, interventions.

In [2]:
df = pd.read_csv('/home/charlie/Downloads/uk_covid.csv', skiprows=1)#.set_index('Code')

In [3]:
start_date = '1 August 2020'
end_date = '9 January 2021'
Ds = pd.date_range(start=start_date, end=end_date)

Rs = df['Unnamed: 2'].unique()
Rs = Rs[7:]
Rs_uk = Rs[:]

CMs = list(df.columns)
CMs = [CM for CM in CMs if CM not in [s for s in CMs if 'Unnamed' in s]]
CMs = CMs[2:]
CMs.remove('Only answer if the local area has at least one university')

nRs = len(Rs)
nDs = len(Ds)
nCMs = len(CMs)

### Extracting dates

Let's make a binary matrix  `active_cms: nRs x nCMs x nDs`, and populate it with the start and end dates

In [4]:
active_CMs_uk = np.zeros((nRs, nCMs, nDs))

Make dataset that contains only local areas and a set of start and end dates

In [5]:
# set columns as the first row:
first_row = list(df.iloc[0])
df.columns = first_row
df = df[['Local area', 'Start date', 'End date']]

Make a function that populates `active_CMs`, given a start-end date pair, and indices corresponding to the region and counter-measure


In [6]:
def populate(start, end, R, CM):
    # convert start and end to indices
    if start == 'nan' : return
    try:
        if math.isnan(start): return
    except: pass
    if start == 'Before 1 August' or start == 'Before 1 August 2020' or start == 'Before August 1' :
        start = '1 August 2020'
    if end == 'no' or end == 'No': end = '9 January 2021'
    try: start = pd.to_datetime(start, format='%d, %M, %Y', infer_datetime_format=True)
    except:
        print('start time conversion failed', i, j)
        return
    try:
        if end == 'January 3': end = '3 January 2021'
    except: pass

    try: end = pd.to_datetime(end, format='%d, %M, %Y', infer_datetime_format=True)
    except:
        print('end time conversion failed', i, j)
        return
    try: start_ind = Ds.get_loc(start)
    except:
        print('getting start loc failed', i, j)
        return
    try: end_ind = Ds.get_loc(end)
    except:
        print('getting end loc failed', i, j)
        return

    active_CMs_uk[R, CM, start_ind:end_ind] = 1
    print('worked')

Loop over rows, identify local area

In [7]:
for i in range(df.shape[0]):
    if df.iloc[i][0] in Rs:
        R_index = list(Rs).index(df.iloc[i][0])
        for j in range(1, nCMs):
            start = df.iloc[i][j]
            end = df.iloc[i][nCMs + j]
            populate(start, end, R_index, j)

worked
worked
worked
start time conversion failed 16 4
worked
getting start loc failed 16 6
worked
worked
worked
worked
start time conversion failed 16 11
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
getting end loc failed 22 4
worked
getting start loc failed 22 6
getting end loc failed 22 7
worked
worked
worked
end time conversion failed 22 13
end time conversion failed 22 14
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
getting end loc failed 24 13
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
getting start loc failed 28 6
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
gett

## Germany

In [8]:
df = pd.read_csv('/home/charlie/Downloads/germany_covid.csv', skiprows=1)#.set_index('Code')

In [9]:
Rs = df['Unnamed: 2'].unique()
Rs = Rs[2:]
nRs = len(Rs)
Rs_germany = Rs[:]

### Extracting dates

Let's make a binary matrix  `active_cms: nRs x nCMs x nDs`, and populate it with the start and end dates

In [10]:
active_CMs_germany = np.zeros((nRs, nCMs, nDs))

Make dataset that contains only local areas and a set of start and end dates

In [11]:
# set columns as the first row:
first_row = list(df.iloc[0])
df.columns = first_row
df = df[['Local area', 'Start date', 'End date']]

Make a function that populates `active_CMs`, given a start-end date pair, and indices corresponding to the region and counter-measure

In [12]:
def populate(start, end, R, CM):
    # convert start and end to indices
    if start == 'nan' : return
    try:
        if math.isnan(start): return
    except: pass
    if start == 'Before 1 August' or start == 'Before 1 August 2020' or start == 'Before August 1' :
        start = '1 August 2020'
    if end == 'no' or end == 'No': end = '9 January 2021'
    try: start = pd.to_datetime(start, format='%d/%m/%Y', infer_datetime_format=True)
    except:
        print('start time conversion failed', i, j)
        return
    try:
        if end == 'January 3': end = '3 January 2021'
    except: pass

    try: end = pd.to_datetime(end, format='%d/%m/%Y', infer_datetime_format=True)
    except:
        print('end time conversion failed', i, j)
        return
    try: start_ind = Ds.get_loc(start)
    except:
        print('getting start loc failed', i, j)
        return
    try: end_ind = Ds.get_loc(end)
    except:
        print('getting end loc failed', i, j)
        return

    active_CMs_germany[R, CM, start_ind:end_ind] = 1
    print('worked')

Loop over rows, identify local area

In [13]:
for i in range(df.shape[0]):
    if df.iloc[i][0] in Rs:
        R_index = list(Rs).index(df.iloc[i][0])
        for j in range(1, nCMs):
            start = df.iloc[i][j]
            end = df.iloc[i][nCMs + j]
            populate(start, end, R_index, j)

getting start loc failed 9 1
getting start loc failed 9 2
getting start loc failed 9 3
getting start loc failed 9 4
getting start loc failed 9 6
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
getting end loc failed 10 11
worked
getting end loc failed 10 14
worked
getting end loc failed 11 1
worked
worked
worked
getting end loc failed 11 14
worked
getting end loc failed 12 1
getting end loc failed 12 2
getting end loc failed 12 3
worked
getting end loc failed 13 1
worked
worked
worked
getting end loc failed 14 1
worked
worked
getting end loc failed 14 4
getting end loc failed 15 3
worked
getting end loc failed 16 3
worked
getting end loc failed 17 3
worked
getting end loc failed 18 3
worked
getting start loc failed 19 1
getting start loc failed 19 2
getting start loc failed 19 3
getting start loc failed 19 4
getting start loc failed 19 6
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
getting end loc 

## Austria

In [14]:
df = pd.read_csv('/home/charlie/Downloads/austria_covid.csv', skiprows=1)#.set_index('Code')


In [15]:
Rs = df['Unnamed: 1'].unique()
Rs = Rs[1:]
Rs = np.delete(Rs, 1)
nRs = len(Rs)
Rs_austria = Rs[:]

### Extracting dates

Let's make a binary matrix  `active_cms: nRs x nCMs x nDs`, and populate it with the start and end dates

In [16]:
active_CMs_austria = np.zeros((nRs, nCMs, nDs))

Make dataset that contains only local areas and a set of start and end dates

In [17]:
# set columns as the first row:
first_row = list(df.iloc[0])
df.columns = first_row
df = df[['Region', 'Start date', 'End date']]

Make a function that populates `active_CMs`, given a start-end date pair, and indices corresponding to the region and counter-measure


In [18]:
def populate(start, end, R, CM):
    # convert start and end to indices
    if start == 'nan' : return
    try:
        if math.isnan(start): return
    except: pass
    if start == 'Before 1 August' or start == 'Before 1 August 2020' or start == 'Before August 1' :
        start = '1 August 2020'
    if end == 'no' or end == 'No': end = '9 January 2021'
    try: start = pd.to_datetime(start, format='%d, %M, %Y', infer_datetime_format=True)
    except:
        print('start time conversion failed', i, j)
        return
    try:
        if end == 'January 3': end = '3 January 2021'
    except: pass

    try: end = pd.to_datetime(end, format='%d, %M, %Y', infer_datetime_format=True)
    except:
        print('end time conversion failed', i, j)
        return
    try: start_ind = Ds.get_loc(start)
    except:
        print('getting start loc failed', i, j)
        return
    try: end_ind = Ds.get_loc(end)
    except:
        print('getting end loc failed', i, j)
        return

    active_CMs_austria[R, CM, start_ind:end_ind] = 1
    print('worked')

Loop over rows, identify local area

In [19]:
for i in range(df.shape[0]):
    if df.iloc[i][0] in Rs:
        R_index = list(Rs).index(df.iloc[i][0])
        for j in range(1, nCMs):
            start = df.iloc[i][j]
            end = df.iloc[i][nCMs + j]
            populate(start, end, R_index, j)


worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
start time conversion failed 1 13
start time conversion failed 1 14
start time conversion failed 1 15
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
start time conversion failed 11 13
start time conversion failed 11 14
start time conversion failed 11 15
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
start time conversion failed 20 13
start time conversion failed 20 14
start time conversion failed 20 15
worked
worked
wo

## Italy

In [20]:
df = pd.read_csv('/home/charlie/Downloads/italy_covid.csv', skiprows=1)#.set_index('Code')


In [21]:
Rs = df['Unnamed: 1'].unique()
Rs = Rs[7:]
nRs = len(Rs)
Rs_italy = Rs[:]

### Extracting dates

Let's make a binary matrix  `active_cms: nRs x nCMs x nDs`, and populate it with the start and end dates

In [22]:
active_CMs_italy = np.zeros((nRs, nCMs, nDs))

Make dataset that contains only local areas and a set of start and end dates

In [23]:
# set columns as the first row:
first_row = list(df.iloc[0])
df.columns = first_row
df = df[['Region', 'Start date', 'End date']]

Make a function that populates `active_CMs`, given a start-end date pair, and indices corresponding to the region and counter-measure


In [24]:
def populate(start, end, R, CM):
    # convert start and end to indices
    if start == 'nan' : return
    try:
        if math.isnan(start): return
    except: pass
    if start == 'Before 1 August' or start == 'Before 1 August 2020' or start == 'Before August 1' :
        start = '1 August 2020'
    if end == 'no' or end == 'No': end = '9 January 2021'
    try: start = pd.to_datetime(start, format='%d, %M, %Y', infer_datetime_format=True)
    except:
        print('start time conversion failed', i, j)
        return
    try:
        if end == 'January 3': end = '3 January 2021'
    except: pass

    try: end = pd.to_datetime(end, format='%d, %M, %Y', infer_datetime_format=True)
    except:
        print('end time conversion failed', i, j)
        return
    try: start_ind = Ds.get_loc(start)
    except:
        print('getting start loc failed', i, j)
        return
    try: end_ind = Ds.get_loc(end)
    except:
        print('getting end loc failed', i, j)
        return

    active_CMs_italy[R, CM, start_ind:end_ind] = 1
    print('worked')

Loop over rows, identify local area

In [25]:
for i in range(df.shape[0]):
    if df.iloc[i][0] in Rs:
        R_index = list(Rs).index(df.iloc[i][0])
        for j in range(1, nCMs):
            start = df.iloc[i][j]
            end = df.iloc[i][nCMs + j]
            populate(start, end, R_index, j)

getting end loc failed 9 1
getting end loc failed 9 2
getting end loc failed 9 4
start time conversion failed 9 5
worked
worked
worked
worked
worked
end time conversion failed 9 11
worked
end time conversion failed 9 14
getting end loc failed 9 15
getting end loc failed 10 1
worked
getting end loc failed 10 6
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
worked
getting end loc failed 14 1
getting end loc failed 14 2
getting end loc failed 14 4
start time conversion failed 14 5
worked
worked
worked
worked
end time conversion failed 14 11
worked
end time conversion failed 14 14
getting start loc failed 15 7
getting end loc failed 16 1
worked
getting end loc failed 16 6
worked
worked
worked
worked
worked
worked
worked
worked
worked
getting end loc failed 20 1
getting end loc failed 20 2
getting end loc failed 20 4
start time conversion failed 20 5
start time conversion failed 20 6
worked
worked
worked
worked
end time conversion failed 20 11
worked
end time conversi

Putting the countries together

In [26]:
# regions/local areas
Rss = np.concatenate((Rs_uk, Rs_germany, Rs_austria, Rs_italy))
# combined active_CMs
active_CMs = np.concatenate((active_CMs_uk, active_CMs_germany, active_CMs_austria, active_CMs_italy))
